In [4]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.figsize'] = [12.0, 8.0]
mpl.rcParams['font.size'] = 16

In [5]:
# xyzvws
class Parameters(object):
    def __init__(self, n, n_procs):
        self.n = np.asarray(n)
        self.n_procs = np.asarray(n_procs)
        self.l = self.n / self.n_procs
        self.n_fields = 2
        
    def size_6d(self):
        return 16 * np.prod(self.l)
    
    def size_exz(self):
        return self.size_6d() / self.l[2] * 2
        
    def size_exv(self):
        return self.size_6d() / self.l[3] * 2
    
    def size_moments(self):
        return self.l[0] * self.l[1] * self.l[2] * self.n_fields
    
    def t_erkstandard(self):
        return sum(self.t_erkstage(stage) for stage in range(4))
    
    def t_erkstage(self, stage):
        return self.t_rkupd() + self.t_calcaux() + self.t_calcrhs() + self.t_last()
    
    def t_rkupd(self):
        return (0.213 * (self.l[0] * self.l[1] * self.l[2] * self.l[3] * self.l[4] * self.l[5]) 
                / (70*16*8*24*8*1))
    
    # calcaux
    def t_calcaux(self):
        return (self.t_momlocal() + self.t_momallreduce() + self.t_fldsolve() + self.t_baremf() + 
                self.t_calcdf1() + self.t_hfromf())
    
    def t_momlocal(self):
        return .237 * self.size_6d() / 27.6e6
    
    def t_momallreduce(self):
        # FIXME * log(n_procs_vwspec)
        return 2.084 * self.l[0] * self.l[1] * self.l[2] / (280*16*8)
    
    def t_fldsolve(self):
        return 2.671 * self.l[0]**2 * self.l[1] * self.l[2] / (280**2*16*8)
    
    def t_baremf(self):
        return self.l[0] * self.l[1] * self.l[2] * self.l[4] * self.l[5] * .370 / (70*16*8*8*1)
    
    def t_calcdf1(self):
        return self.t_compute_f() + self.t_df1_exz() + self.t_df1_exv()
    
    def t_compute_f(self):
        return .095 * self.size_6d() / 27.5e6
    
    def t_df1_exz(self):
        return .577 * (self.l[0] * self.l[1] * self.l[3] * self.l[4] * self.l[5]) / (280*16*24*2*1)
    
    def t_df1_exv(self):
        return .723 * (self.l[0] * self.l[1] * self.l[2] * self.l[4] * self.l[5]) / (280*16*8*2*1)
    
    def t_hfromf(self):
        return (.150 * (self.l[0] * self.l[1] * self.l[2] * self.l[3] * self.l[4] * self.l[5]) 
                / (70*16*8*24*8*1))
    
    # calcrhs
    def t_calcrhs(self):
        return (4.753 * (self.l[0] * self.l[1] * self.l[2] * self.l[3] * self.l[4] * self.l[5]) 
                / (70*16*8*24*8*1))
    
    def t_last(self):
        return (0.122 * (self.l[0] * self.l[1] * self.l[2] * self.l[3] * self.l[4] * self.l[5]) 
                / (70*16*8*24*8*1))
        
    def __repr__(self):
        return ("""
n = {}, n_procs = {} l = {}
size_6d = {} MB, size_exz = {} MB, size_exv = {} MB
"""
            .format(self.n, self.n_procs, self.l, self.size_6d() / 1e6,
               self.size_exz() / 1e6, self.size_exv() / 1e6))


In [6]:
def model(prm, t):
    print("""
t_erk_standard       = {:.3f} ms / {:.3f} ms
  t_erk_stage3       = {:.3f} ms / {:.3f} ms
    t_calcaux        = {:.3f} ms / {:.3f} ms
      t_momlocal     = {:.3f} ms / {:.3f} ms
      t_momallreduce = {:.3f} ms / {:.3f} ms
      t_fldsolve     = {:.3f} ms / {:.3f} ms
      t_baremf       = {:.3f} ms / {:.3f} ms
      t_calcdf1      = {:.3f} ms / {:.3f} ms
        t_compute_f  = {:.3f} ms / {:.3f} ms
        t_df1_exz    = {:.3f} ms / {:.3f} ms
        t_df1_exv    = {:.3f} ms / {:.3f} ms
      t_hfromf       = {:.3f} ms / {:.3f} ms
    t_calcrhs        = {:.3f} ms / {:.3f} ms
    """.format(prm.t_erkstandard(), t["erkstandard"],
               prm.t_erkstage(2), t["erkstage3"],
               prm.t_calcaux(), t["calcaux"],
               prm.t_momlocal(), t["momlocal"],
               prm.t_momallreduce(), t["momallreduce"],
               prm.t_fldsolve(), t["fldsolve"],
               prm.t_baremf(), t["baremf"],
               prm.t_calcdf1(), t["calcdf1"],
               prm.t_compute_f(), t["compute_f"],
               prm.t_df1_exz(), t["df1_exz"],
               prm.t_df1_exv(), t["df1_exv"],
               prm.t_hfromf(), t["hfromf"],
               prm.t_calcrhs(), t["calcrhs"]))

In [7]:
prm_i120 = Parameters([120, 8, 36, 32, 14, 2], [1, 1, 3, 2, 1, 1])
t_i120 = {"erkstandard" : 78.257,
        "erkstage3" : 19.342,
          "calcaux" : 6.103,
            "momlocal": .740,
            "momallreduce": .245,
            "fldsolve" : .622,
            "baremf" : .370,
            "calcdf1" : 3.404,
            "compute_f" : .238,
            "df1_exz" : 1.159,
            "df1_exv" : 2.003,
            "hfromf" : .150,
           "calcrhs" : 12.123
        }
print(prm_i120)
model(prm_i120, t_i120)


n = [120   8  36  32  14   2], n_procs = [1 1 3 2 1 1] l = [120.   8.  12.  16.  14.   2.]
size_6d = 82.57536 MB, size_exz = 13.76256 MB, size_exv = 10.32192 MB


t_erk_standard       = 95.275 ms / 78.257 ms
  t_erk_stage3       = 23.819 ms / 19.342 ms
    t_calcaux        = 8.555 ms / 6.103 ms
      t_momlocal     = 0.709 ms / 0.740 ms
      t_momallreduce = 0.670 ms / 0.245 ms
      t_fldsolve     = 0.368 ms / 0.622 ms
      t_baremf       = 1.665 ms / 0.370 ms
      t_calcdf1      = 4.693 ms / 3.404 ms
        t_compute_f  = 0.285 ms / 0.238 ms
        t_df1_exz    = 1.154 ms / 1.159 ms
        t_df1_exv    = 3.254 ms / 2.003 ms
      t_hfromf       = 0.450 ms / 0.150 ms
    t_calcrhs        = 14.259 ms / 12.123 ms
    


In [8]:
prm_32a = Parameters([280, 16, 24, 48, 32, 2], [1, 1, 3, 2, 16, 2])
t_32a = {"erkstandard" : 49.107,
        "erkstage3" : 11.995,
          "calcaux" : 6.696,
            "momlocal" : .227,
            "momallreduce" : 2.084,
            "fldsolve" : 2.618,
            "baremf" : .357,
            "calcdf1" : 1.400,
            "compute_f" : .095,
            "df1_exz" : .577,
            "df1_exv" : .723,
            "hfromf" : .160,
          "calcrhs" : 4.826
        }
print(prm_32a)
model(prm_32a, t_32a)


n = [280  16  24  48  32   2], n_procs = [ 1  1  3  2 16  2] l = [280.  16.   8.  24.   2.   1.]
size_6d = 27.52512 MB, size_exz = 6.88128 MB, size_exv = 2.29376 MB


t_erk_standard       = 47.978 ms / 49.107 ms
  t_erk_stage3       = 11.994 ms / 11.995 ms
    t_calcaux        = 6.906 ms / 6.696 ms
      t_momlocal     = 0.236 ms / 0.227 ms
      t_momallreduce = 2.084 ms / 2.084 ms
      t_fldsolve     = 2.671 ms / 2.618 ms
      t_baremf       = 0.370 ms / 0.357 ms
      t_calcdf1      = 1.395 ms / 1.400 ms
        t_compute_f  = 0.095 ms / 0.095 ms
        t_df1_exz    = 0.577 ms / 0.577 ms
        t_df1_exv    = 0.723 ms / 0.723 ms
      t_hfromf       = 0.150 ms / 0.160 ms
    t_calcrhs        = 4.753 ms / 4.826 ms
    


In [9]:
prm_32b = Parameters([280, 16, 24, 48, 32, 2], [1, 1, 6, 2, 8, 2])
t_32b = {"erkstandard" : 68.402,
        "erkstage3" : 16.465,
          "calcaux" : 9.862,
            "momlocal" : .237,
            "momallreduce" : 1.052,
            "fldsolve" : 2.587,
            "baremf" : .357,
            "calcdf1" : 5.304,
            "compute_f" : .096,
            "df1_exz" : 1.608,
            "df1_exv" : 3.596,
            "hfromf" : .160,
          "calcrhs" : 6.127
        }
print(prm_32b)
model(prm_32b, t_32b)


n = [280  16  24  48  32   2], n_procs = [1 1 6 2 8 2] l = [280.  16.   4.  24.   4.   1.]
size_6d = 27.52512 MB, size_exz = 13.76256 MB, size_exv = 2.29376 MB


t_erk_standard       = 40.776 ms / 68.402 ms
  t_erk_stage3       = 10.194 ms / 16.465 ms
    t_calcaux        = 5.106 ms / 9.862 ms
      t_momlocal     = 0.236 ms / 0.237 ms
      t_momallreduce = 1.042 ms / 1.052 ms
      t_fldsolve     = 1.335 ms / 2.587 ms
      t_baremf       = 0.370 ms / 0.357 ms
      t_calcdf1      = 1.972 ms / 5.304 ms
        t_compute_f  = 0.095 ms / 0.096 ms
        t_df1_exz    = 1.154 ms / 1.608 ms
        t_df1_exv    = 0.723 ms / 3.596 ms
      t_hfromf       = 0.150 ms / 0.160 ms
    t_calcrhs        = 4.753 ms / 6.127 ms
    


In [309]:
prm_512 = Parameters([560, 32, 24, 96, 64, 2], [1, 1, 6, 4, 64, 2])
t_512 = {"erkstandard" : 115.,
        "erkstage3" : 27.297,
          "calcaux" : 20.868,
            "ccdens" : 3.665,
            "fldsolve" : 10.243,
            "baremf" : .382,
            "calcdf1" : 6.316,
            "hfromf" : .231,
          "calcrhs" : 5.924
        }
print(prm_512)
model(prm_512, t_512)


n = [560  32  24  96  64   2]
n_procs = [ 1  1  6  4 64  2]
l = [560.  32.   4.  24.   1.   1.]
size_6d = 27.52512 MB


t_erk_standard   = 86.144 ms / 115.000 ms
  t_erk_stage3   = 21.536 ms / 27.297 ms
    t_calcaux    = 16.448 ms / 20.868 ms
      t_ccdens   = 3.959 ms / 3.665 ms
      t_fldsolve = 10.684 ms / 10.243 ms
      t_baremf   = 0.370 ms / 0.382 ms
      t_calcdf1  = 1.285 ms / 6.316 ms
      t_hfromf   = 0.150 ms / 0.231 ms
    t_calcrhs    = 4.753 ms / 5.924 ms
    
